In [ ]:
!pip uninstall libav-tools
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install pyaudio

In [ ]:
!pip install ffmpeg-python

In [ ]:
pip install  soundfile 

In [ ]:
import os
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
import warnings
import pandas as pd
import csv
import tensorflow as tf
import soundfile as sf
import difflib
import IPython.display as ipd
import librosa
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
from keras.regularizers import l1
warnings.filterwarnings("ignore")

In [ ]:
labels = []
with open('/content/drive/My Drive/all_data.csv', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = csv.reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        labels.append(row[1])
labels.remove('label')

print(len(labels))

In [ ]:
all_wave = np.load("/content/drive/My Drive/50all_wave.npy", allow_pickle=True) 
all_label = np.load("/content/drive/My Drive/50all_label.npy", allow_pickle=True) 
print(len(all_label))
print(len(all_wave))

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y=le.fit_transform(all_label)
classes= list(le.classes_)
print(y.shape)

In [ ]:
from keras.utils import np_utils
y=np_utils.to_categorical(y, num_classes=len(labels))
print(y.shape)

In [ ]:
all_wave = np.array(all_wave).reshape(-1,884  ,1)
print(all_wave.shape)

In [ ]:
all_waves = np.array(all_wave)
y = np.array(y)
all_waves = all_waves.reshape(72100  ,884 ,1)
all_wave = all_wave.reshape(all_waves.shape)
all_waves = all_waves.transpose()
all_waves = all_waves.reshape((884,72100 ,1))
print(all_waves.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test  = train_test_split(all_waves,y,  test_size = 0.2,random_state=777,shuffle=True)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf
from tensorflow import keras
from keras.regularizers import l1 

# instantiate regularizer
reg = l1(0.001)

K.clear_session()

inputs = Input(shape=(72100, 1))

#First Conv1D layer
conv = Conv1D(8,13, padding='valid', activation='relu', strides=1)(inputs)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Second Conv1D layer
conv = Conv1D(16, 11, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Third Conv1D layer
conv = Conv1D(32, 9, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Fourth Conv1D layer
conv = Conv1D(64, 7, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Flatten layer
conv = Flatten()(conv)

#Dense Layer 1
conv = Dense(256, activation='relu' ,activity_regularizer=l1(0.001))(conv)
conv = Dropout(0.3)(conv)

#Dense Layer 2
conv = Dense(128, activation='relu' ,activity_regularizer=l1(0.001))(conv)
conv = Dropout(0.3)(conv)

outputs = Dense(len(labels), activation='softmax')(conv)

model = Model(inputs, outputs)
model.summary()

In [ ]:
filepath = "/content/drive/My Drive/model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
X_train = tf.convert_to_tensor(X_train, np.float32)
y_train = tf.convert_to_tensor(y_train, np.float32)
X_test = tf.convert_to_tensor(X_test, np.float32)
y_test = tf.convert_to_tensor(y_test, np.float32)
history= model.fit(X_train, y_train ,epochs=300, batch_size=128, validation_data=(X_test,y_test),callbacks=callbacks_list)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.0001) 
mc = ModelCheckpoint('model.h5', monitor='val_acc', verbose=1, save_best_only= True ,mode='max')

In [ ]:
from matplotlib import pyplot
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
from keras.models import load_model
model=load_model('/content/drive/My Drive/model.h5')

In [ ]:
def predict(audio):
    prob=model.predict(audio.reshape(1,72100,1))
    index=np.argmax(prob[0])
    return classes[index]

In [ ]:
import random
index=random.randint(0,len(X_test)-1)
#samples = X_test[index].ravel()
samples = np.ravel(X_test[index])
print("Audio:",classes[np.argmax(y_test[index])])
ipd.Audio(samples, rate=16000)

In [ ]:
b = print("Text:",predict(samples))

In [ ]:
a = predict(samples)
b = ['و رجح التقرير الذي أعده معهد أبحاث هضبة التبت في الأكاديمية الصينية للعلوم أن تستمر درجات الحرارة و مستويات الرطوبة في الارتفاع طوال هذا القرن مما قد يؤدي إلى تراجع مساحات الأنهار الجليدية وانتشارِ التصحر و ذكر التقرير أن تراجع مساحة الجليد يمكن أن يخلى بمعدلات امدادات المياه لعدد من أنهار آسيا الرئيسية التي تنبع من الهضبة' , 'يرتبط الورد بين الناس بصباحات و المساءات و الهدايا و أما عند أهل الطائف فهو ارتباط روحي و تاريخي و عشق متبادل و يسقونه و يعرفونه  كما يعرفهم منذ القرن التاسع الهجري و أعبقها ما يروى من مياه المطر وفي كل عاما من موسم الورد الطائفي نهاية آذار و أوائل نيسان تروى قصة حب بينه و بين أهله على قمم و سفوح جبال الطائف' , 'مجموعة من علماء النفس الأمريكيين توصلوا لسبب ذلك وهو أن دموع الفرح قد تكون طريقة الجسم الصحيحة لاستعادة التوازن العاطفي بهذه التعبيرات' , 'و الشوكولاتة هي أسرع في الذوبان من الزبدة و تحتاج فقط لدرجة حرارة انصهار تبلغ حوالي ثلاثة و ثلاثين درجة مئوية أي أقل من درجة حرارة الإنسان لذلك تذوب الشوكولاتة بسهولة على اللسان بحرارة اللعاب فقط' , 'و يضم المعرض نسخا نادرة من المصاحف المذهبة و أخرى مخطوطة لا يتجاوز حجمها كف اليد و أخرى كبيرة تزيد مساحة ضلع الصفحة فيها عن خمسة و ستين سنتمتر  فضلا عن نسخ مترجمة لمعاني القرآن كتبت بحروف بريل الإنكليزية للقراء المكفوفين' , 'تقوم على إطلاق مئات الأقمار الصناعية المصغرة إلى مدار منخفض حول الأرض بحيث يتولى كل قمر صناعي باستقبال البيانات من عدة مئات من المحطات الأرضية ثم إعادة بثها عبر موجات إذاعية عريضة  إلى الهواتف المنتشرة في العالم' , 'كشفت دراسة أمريكية أن المشاعر الأساسية الستة سعيد و حزين و خائف و غضبان و مندهش و مشمئز لا تغطي كل التعابير التي تظهر على الوجه كما كان يعتقد و بعضها معقد مثل مشمئز بسعادة و غضبان بحزن' , 'يعد العلاج بالأحجار الكريمة من طرق السائدة في الشرق القديم حيث كان العرب أول من عرف قيمتها العلاجية في القرن العاشر قبل الميلاد في مملكة سبأ فطلبها الفراعنة للزينة و التبرك والعلاج' , 'طور علماء هولنديون أول بديل فعال عن المضادات الحيوية و هو ما يعتبر تقدم هام في مكافحة العدوى المقاومة للأدوية و يأمل الباحثون تطوير حبوب من الدواء خلال خمسة سنوات' , 'تسعى السعودية إلى تحقيق طفرة في مستقبل الرعاية الصحية بالمملكة من خلال مشروع جديد لرسم خرائط الجينوم البشري يستمر خمس سنوات لتكوين قاعدة بيانات للحمض النووي ليستخدمها العلماء في مختلف الأغراض' , 'أفادت دراسة جديدة بأن الصيام لمدة أقل من ثلاثة أيام يمكن أن يجدد نشاط الجهاز المناعي بأكمله حتى في كبار السن و أشارت الدراسة إلى أن تجويع الجسم يحفز الخلايا الجذعية على تجديد خلايا الدم البيضاء المقاومة  للعدوى مما يزيد صعوبة مقاومتهم للأمراض الشائعة' , 'و يقوم هذا التصميم على التخفيف من ازدحام الأيقونات و الخيارات و التركيز على الخريطة و تسهيل عملية استكشاف الأماكن الجديدة و تدعم النسخة الجديدة من خرائط جوجل إظهار معلومات الطريق في الوقت الحقيقي و كما يقترح التطبيق و بشكل فوري طريق بديل لمساعدة المستخدم في تفادي الازدحام' , 'و قال العلماء إن مثل هذه الأفكار تزيد فهمهم لكيفية تعبير الناس عن عواطفهم و مراقبتها و لما هذا من ارتباط مهم بالصحة النفسية و الجسدية و نوعية العلاقات مع الآخرين بل حتى كيفية تعامل الناس مع بعضهم بشكل جيد' , 'و يقول باحثون إن هذه الدراسة تظهر الحاجة لمراجعة التوجيهات الإرشادية الحالية التي تنصح بإنقاص الوزن بخطى بطيئة و ثابتة و يقول كبير الباحثين في الفريق  إن السمنة المفرطة ليست مرض نابع من نمط الحياة فحسب وفق الاعتقاد الشائع' , 'تثير السيجارة الإلكترونية الكثير من الجدل بين كونها وسيلة صحية لتخلص من التدخين أو بضاعة تجارية يعمل منتجوها على تحسينها باستمرار للفوز برضى الزبائن' , 'التوحد هو مجموعة اضطرابات عصبية في التطور تشمل أعراضها وجود مشاكل في السلوك الاجتماعي للشخص المصاب' , 'و يستخدم البرنامج نظام سحابي للذكاء الاصطناعي يسمح له بتحليل الإيماءات و التعابير' , 'النكت كلمة عامية تقال عند الكلام  المضحك بطريقة جميلة يملأها المرح  وأجمل  ما في النكت بأنها تجعلنا نبتسم ونضحك و لذلك لها أثر نفسي كبير جدا علينا فحسب بعض الدراسات العلمية بأن الضحك يقي من الاكتئاب ويقوي عضلات القلب وغيرها الكثير من الفوائد التي تعود بالمنفعة على جسم الإنسان' , 'كان المستكشف الأوربي كريستوفر كولومبوس مندهشا عندما شاهد شجرة الشوكولاتة لأول مرة في القارة الأمريكية موطنها الأصلي و كانت تستخدم فقط كمشروب في ذلك الوقت' , 'معالجة اللغة الطبيعية هو مجال يعمل على بناء خوارزميات حسابية لتحليل اللغة البشرية وتمثيلها تلقائيا وقد مكنت النظم القائمة على مجموعة واسعة من التطبيقات مثل محرك البحث القوي من جوجل  ومؤخرا المساعد الصوتي أليكسا من أمازون']

print(a)

from difflib import SequenceMatcher
#def Define_the_error (a ,b):
def similar(a, b):
  return SequenceMatcher(None, a, b).ratio()

error =[]
w = []
for i in b:
  if similar(a, i) >0.7:
    # print(similar(a, i), i)

    cases =[(a ,i)]
    for i, s in enumerate (difflib.ndiff(a ,i)):
      if s[0] == '': continue 
      elif s[0] == '+':
        w = u'' "{}" ''.format(s[-1],i)
        error.append(w)
        error = list(dict.fromkeys(error))
        

print(error)

 


In [ ]:
def enumerate(sequence, start=0):
    n = start
    for elem in sequence:
        yield n, elem
        n += 1

In [ ]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
def stereoToMono(audiodata):
  newaudiodata = []

  for i in range(len(audiodata)):
      d = (audiodata[i][0] + audiodata[i][1])/2
      newaudiodata.append(d)

  return np.array(newaudiodata, dtype='float32')


In [ ]:
mfcc =[]
mfcc_audio = []

letter ={'ب':'/content/drive/My Drive/lett/refernc/silence_baa.wav','ت':'/content/drive/My Drive/lett/refernc/silence_ta.wav','ث':'/content/drive/My Drive/lett/refernc/silence_thaa.wav','ج':'/content/drive/My Drive/lett/refernc/silence_gem.wav','ح':'/content/drive/My Drive/lett/refernc/silence_haa.wav','خ':'/content/drive/My Drive/lett/refernc/silence_kaa.wav','د':'/content/drive/My Drive/lett/refernc/silence_dal.wav','ذ':'/content/drive/My Drive/lett/refernc/silence_thal.wav','ر':'/content/drive/My Drive/lett/refernc/silence_raa.wav','ز':'/content/drive/My Drive/lett/refernc/silence_zie.wav','س':'/content/drive/My Drive/lett/refernc/silence_sen.wav','ش':'/content/drive/My Drive/lett/refernc/silence_shen.wav','ص':'/content/drive/My Drive/lett/refernc/silence_sad.wav','ض':'/content/drive/My Drive/lett/refernc/silence_dad.wav','ط':'/content/drive/My Drive/lett/refernc/silence_taa.wav','ظ':'/content/drive/My Drive/lett/refernc/silence_thdaa.wav','ع':'/content/drive/My Drive/lett/refernc/silence_aen.wav','غ':'/content/drive/My Drive/lett/refernc/silence_gen.wav','ف':'/content/drive/My Drive/lett/refernc/silence_faa.wav','ق':'/content/drive/My Drive/lett/refernc/silence_kaf.wav','ك':'/content/drive/My Drive/lett/refernc/silence_kefa.wav','ل':'/content/drive/My Drive/lett/refernc/silence_lam.wav','م':'/content/drive/My Drive/lett/refernc/silence_mem.wav','ن':'/content/drive/My Drive/lett/refernc/silence_non.wav','ه':'/content/drive/My Drive/lett/refernc/silence_hhaa.wav','و':'/content/drive/My Drive/lett/refernc/silence_wow.wav','ي':'/content/drive/My Drive/lett/refernc/silence_eaa.wav'}
lis = error

#lis.append(lis)
new_lis = []
for k in lis:
    if k in letter:
        new_lis.append(letter[k])
        print(new_lis)

print(new_lis)

#for i in new_lis:
for index, item in enumerate(new_lis):
      next = index + 1
      if next < len(new_lis):
          print (index, item, next)
  
          a = ipd.Audio(item ,autoplay=True)
          ipd.display(a)
          sample ,sample_rate = librosa.load(item ,sr =16000 ,mono=False)
          audio, sr = get_audio()
          audio = stereoToMono(audio)

          if len(sample) > len(audio) :
            dif = len(sample) - len(audio)

          else:
            dif = len(audio) - len(sample)
          sample=np.pad(sample, (0, dif), 'constant', constant_values=0)
          mfcc = librosa.feature.mfcc(y =sample ,sr =16000 ,n_mfcc=12)


          mfcc_audio = librosa.feature.mfcc(y =audio ,sr =16000 ,n_mfcc=12)

          print(mfcc.shape ,mfcc_audio.shape)

          hop_size = 2205

          D, wp = librosa.core.dtw(X=mfcc, Y=mfcc_audio, metric='euclidean')
          wp_s = np.asarray(wp) * hop_size / sample_rate
          best_cost = D[wp[-1, 0], wp[-1, 1]]
          print(best_cost )

          if best_cost <= 500:
            print('لفظ صحيح' + item)

            

          elif best_cost >= 500 :
            

            a = ipd.Audio(new_lis[next] ,autoplay=True)
            ipd.display(a)
            sample ,sample_rate = librosa.load(new_lis[next] ,sr =16000 ,mono=False)
            audio, sr = get_audio()
            audio = stereoToMono(audio)

            if len(sample) > len(audio) :
              dif = len(sample) - len(audio)

            else:
              dif = len(audio) - len(sample)
            sample=np.pad(sample, (0, dif), 'constant', constant_values=0)
            mfcc = librosa.feature.mfcc(y =sample ,sr =16000 ,n_mfcc=12)


            mfcc_audio = librosa.feature.mfcc(y =audio ,sr =16000 ,n_mfcc=12)

            print(mfcc.shape ,mfcc_audio.shape)

            hop_size = 2205

            D, wp = librosa.core.dtw(X=mfcc, Y=mfcc_audio, metric='euclidean')
            wp_s = np.asarray(wp) * hop_size / sample_rate
            best_cost = D[wp[-1, 0], wp[-1, 1]]
            print(best_cost )

In [ ]:
audio, sr = get_audio()

In [ ]:
audio = stereoToMono(audio)

In [ ]:
filename = '/content/prateek_voice_v2.wav'
sf.write(filename, audio, sr)

In [ ]:
filepath='/content/prateek_voice_v2.wav'

In [ ]:
#reading the voice commands
samples, sample_rate = librosa.load(filepath , sr = 16000)
ipd.Audio(samples,rate=16000)


In [ ]:
all_wave =[]
samples, sample_rate = librosa.load('/content/prateek_voice_v2.wav' , mono=True, sr=16000)

#zero padding until data is equal
if len(samples) <738116  : 
  dif = 738116-len(samples)
sample=np.pad(samples, (0, dif), 'constant', constant_values=0)
mfc =librosa.feature.mfcc(y=sample ,sr =16000 ,n_mfcc=50)
all_wave.append(mfc)
print(mfc.shape)
print(sample.shape)

all_wave = np.array(all_wave).reshape(-1,72100   ,1)
print(all_wave.shape)

In [ ]:
#converting voice commands to text
predict(all_wave)